# Sales Data Analysis

This notebook analyzes sales data from Kiosks and Mini-Markets. It identifies the top-selling subcategories, compares unique subcategories between store types, and finds frequently bought-together products.

In [1]:
import pandas as pd

# Load the Excel file
file_path = 'University Dataset 2024.xlsx'
xls = pd.ExcelFile(file_path)

# Read sheets
df_categories = pd.read_excel(xls, 'Categories')
df_stores = pd.read_excel(xls, 'Stores')
df_transactions = pd.read_excel(xls, 'Transactions')

## Data Preprocessing
Merging the transactions dataset with store types to allow for store-specific analysis.

In [2]:
# Merge transactions with store types
df_transactions = df_transactions.merge(df_stores, on='StoreId', how='left')

# Group by store type and subcategory to get total quantities and revenues
df_grouped = df_transactions.groupby(['StoreType', 'Subcategory']).agg(
    total_quantity=('Quantity', 'sum'),
    total_revenue=('Revenue', 'sum')
).reset_index()

## Function to Retrieve Top 10 Subcategories
The function below retrieves the top 10 selling subcategories by quantity for a specified store type.

In [3]:
def get_top_10(store_type):
    ''' Get top 10 selling subcategories by quantity for a given store type '''
    df_filtered = df_grouped[df_grouped['StoreType'] == store_type]
    df_filtered = df_filtered.sort_values(by='total_quantity', ascending=False)
    total_quantity_all = df_filtered['total_quantity'].sum()
    df_filtered['proportion'] = df_filtered['total_quantity'] / total_quantity_all
    return df_filtered.head(10)

## Identifying Top 10 Subcategories
We apply the function to extract the top 10 subcategories for Kiosks and Mini-Markets.

In [8]:
# Get top 10 subcategories for Kiosks and Mini-Markets
kiosk_top_10 = get_top_10('Kiosk')
mini_market_top_10 = get_top_10('Mini-Market')

# Print the top 10 subcategories for Kiosks
print("Top 10 Subcategories for Kiosks:")
print(kiosk_top_10[['Subcategory', 'proportion']])

# Print the top 10 subcategories for Mini-Markets
print("\nTop 10 Subcategories for Mini-Markets:")
print(mini_market_top_10[['Subcategory', 'proportion']])

Top 10 Subcategories for Kiosks:
        Subcategory  proportion
18       CIGARETTES    0.232275
52           HEATED    0.093943
2             BEERS    0.060562
130          WAFERS    0.053521
101  ROLLING PAPERS    0.050865
100   ROLL YOUR OWN    0.039255
74   NATURAL WATER     0.036101
35     ENERGY DRINK    0.035790
23            COLAS    0.034969
19       CIGARILLOS    0.034646

Top 10 Subcategories for Mini-Markets:
        Subcategory  proportion
153      CIGARETTES    0.185428
189          HEATED    0.110687
136           BEERS    0.052088
239  ROLLING PAPERS    0.042758
212  NATURAL WATER     0.037702
238   ROLL YOUR OWN    0.035246
158           COLAS    0.031176
170    ENERGY DRINK    0.028039
176         FILTERS    0.026021
148          CHIPS     0.024266


## Comparing Unique Subcategories
We identify subcategories that appear exclusively in the top 10 list of either Kiosks or Mini-Markets.

In [5]:
# Find unique subcategories in each top-10
kiosk_unique = set(kiosk_top_10['Subcategory']) - set(mini_market_top_10['Subcategory'])
mini_market_unique = set(mini_market_top_10['Subcategory']) - set(kiosk_top_10['Subcategory'])

print('Subcategories in Kiosks top 10 but not in Mini-Markets:', kiosk_unique)
print('Subcategories in Mini-Markets top 10 but not in Kiosks:', mini_market_unique)

Subcategories in Kiosks top 10 but not in Mini-Markets: {'WAFERS', 'CIGARILLOS'}
Subcategories in Mini-Markets top 10 but not in Kiosks: {'FILTERS', 'CHIPS '}


## Analyzing Kiosk-Specific Subcategories
For each unique subcategory in Kiosks, we compare sales proportions and revenue per unit with Mini-Markets.
We also find the top 10 products frequently bought together with these subcategories.

In [6]:
# Analyze kiosk-specific top subcategories
for subcategory in kiosk_unique:
    kiosk_data = kiosk_top_10[kiosk_top_10['Subcategory'] == subcategory]
    mini_market_data = df_grouped[(df_grouped['StoreType'] == 'Mini-Market') & (df_grouped['Subcategory'] == subcategory)]
    
    kiosk_proportion = kiosk_data['proportion'].values[0]
    mini_market_proportion = mini_market_data['total_quantity'].sum() / df_grouped[df_grouped['StoreType'] == 'Mini-Market']['total_quantity'].sum()
    kiosk_avg_revenue = kiosk_data['total_revenue'].values[0] / kiosk_data['total_quantity'].values[0]
    mini_market_avg_revenue = mini_market_data['total_revenue'].sum() / mini_market_data['total_quantity'].sum() if mini_market_data['total_quantity'].sum() > 0 else 0
    
    print(f'Subcategory: {subcategory}')
    print(f'  - Kiosk proportion: {kiosk_proportion:.4f}')
    print(f'  - Mini-Market proportion: {mini_market_proportion:.4f}')
    print(f'  - Kiosk avg revenue per unit: {kiosk_avg_revenue:.2f}')
    print(f'  - Mini-Market avg revenue per unit: {mini_market_avg_revenue:.2f}')
    
    # Find top 10 most frequent products bought together
    invoices_with_subcategory = df_transactions[(df_transactions['StoreType'] == 'Kiosk') & (df_transactions['Subcategory'] == subcategory)]['InvoiceGlobalId']
    df_combined = df_transactions[(df_transactions['StoreType'] == 'Kiosk') & (df_transactions['InvoiceGlobalId'].isin(invoices_with_subcategory))]
    top_combined = df_combined[df_combined['Subcategory'] != subcategory]['Subcategory'].value_counts().head(10)
    print()
    print('  - Top 10 products bought together:')
    print(top_combined)
    print()

Subcategory: WAFERS
  - Kiosk proportion: 0.0535
  - Mini-Market proportion: 0.0106
  - Kiosk avg revenue per unit: 0.55
  - Mini-Market avg revenue per unit: 0.91

  - Top 10 products bought together:
Subcategory
CIGARETTES        2100
CHEWING GUMS      1941
COLAS             1407
CROISSANT         1387
ENERGY DRINK      1270
NATURAL WATER     1047
DESSERT            790
CAKE               710
COUNTLINES         581
FLAVORS            545
Name: count, dtype: int64

Subcategory: CIGARILLOS
  - Kiosk proportion: 0.0346
  - Mini-Market proportion: 0.0233
  - Kiosk avg revenue per unit: 2.08
  - Mini-Market avg revenue per unit: 2.26

  - Top 10 products bought together:
Subcategory
CIGARETTES                  1090
BEERS                        435
REST TOBACCO ACCESSORIES     326
HEATED                       249
COLAS                        247
WAFERS                       236
ROLLING PAPERS               230
FILTERS                      178
ENERGY DRINK                 175
ROLL YOUR OWN 

In [7]:
# Analyze mini-market-specific top subcategories
for subcategory in mini_market_unique:
    mini_market_data = mini_market_top_10[mini_market_top_10['Subcategory'] == subcategory]
    kiosk_data = df_grouped[(df_grouped['StoreType'] == 'Kiosk') & (df_grouped['Subcategory'] == subcategory)]
    
    mini_market_proportion = mini_market_data['proportion'].values[0]
    kiosk_proportion = kiosk_data['total_quantity'].sum() / df_grouped[df_grouped['StoreType'] == 'Kiosk']['total_quantity'].sum()
    mini_market_avg_revenue = mini_market_data['total_revenue'].values[0] / mini_market_data['total_quantity'].values[0]
    kiosk_avg_revenue = kiosk_data['total_revenue'].sum() / kiosk_data['total_quantity'].sum() if kiosk_data['total_quantity'].sum() > 0 else 0
    
    print(f'Subcategory: {subcategory}')
    print(f'  - Mini-Market proportion: {mini_market_proportion:.4f}')
    print(f'  - Kiosk proportion: {kiosk_proportion:.4f}')
    print(f'  - Mini-Market avg revenue per unit: {mini_market_avg_revenue:.2f}')
    print(f'  - Kiosk avg revenue per unit: {kiosk_avg_revenue:.2f}')
    
    # Find top 10 most frequent products bought together
    invoices_with_subcategory = df_transactions[(df_transactions['StoreType'] == 'Mini-Market') & (df_transactions['Subcategory'] == subcategory)]['InvoiceGlobalId']
    df_combined = df_transactions[(df_transactions['StoreType'] == 'Mini-Market') & (df_transactions['InvoiceGlobalId'].isin(invoices_with_subcategory))]
    top_combined = df_combined[df_combined['Subcategory'] != subcategory]['Subcategory'].value_counts().head(10)
    print()
    print('  - Top 10 products bought together:')
    print(top_combined)
    print()

Subcategory: FILTERS
  - Mini-Market proportion: 0.0260
  - Kiosk proportion: 0.0295
  - Mini-Market avg revenue per unit: 0.79
  - Kiosk avg revenue per unit: 0.76

  - Top 10 products bought together:
Subcategory
ROLLING PAPERS              14038
ROLL YOUR OWN               11361
CIGARETTES                   1234
REST TOBACCO ACCESSORIES     1165
BEERS                         622
HEATED                        596
COLAS                         588
ENERGY DRINK                  568
NATURAL WATER                 478
TABLETS                       399
Name: count, dtype: int64

Subcategory: CHIPS 
  - Mini-Market proportion: 0.0243
  - Kiosk proportion: 0.0174
  - Mini-Market avg revenue per unit: 1.94
  - Kiosk avg revenue per unit: 1.79

  - Top 10 products bought together:
Subcategory
EXTRUDED          2068
COLAS             1839
TABLETS           1504
CIGARETTES        1478
BEERS             1344
FLAVORS           1102
HEATED            1072
COUNTLINES         954
ENERGY DRINK       8